In [48]:
#pip install pandas sqlalchemy ipython-sql


In [49]:
#pip install --upgrade prettytable

In [50]:
#pip install --upgrade ipython-sql


In [51]:
# Import required libraries for database connection and data manipulation
import sqlite3
import pandas as pd


In [53]:
# List of CSV file paths
csv_files = {
    "Appearances": r"C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project\Football Database\appearances.csv",
    "Games": r"C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project\Football Database\games.csv",
    "Leagues": r"C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project\Football Database\leagues.csv",
    "Players": r"C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project\Football Database\players.csv",
    "Shots": r"C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project\Football Database\shots.csv",
    "Teams": r"C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project\Football Database\teams.csv",
    "TeamStats": r"C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project\Football Database\teamstats.csv",
}

In [55]:
# Create SQLite database (file-based)
db_file = "football_database.db"
conn = sqlite3.connect(db_file)  # Use a persistent database file


In [56]:
# Load each CSV into a separate table
for table_name, file_path in csv_files.items():
    try:
        # Specify a fallback encoding
        df = pd.read_csv(file_path, encoding='latin1')
        df.to_sql(table_name, conn, index=False, if_exists='replace')
        print(f"Loaded {file_path} into table {table_name}")
    except UnicodeDecodeError as e:
        print(f"Error loading {file_path}: {e}")


Loaded C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project\Football Database\appearances.csv into table Appearances
Loaded C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project\Football Database\games.csv into table Games
Loaded C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project\Football Database\leagues.csv into table Leagues
Loaded C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project\Football Database\players.csv into table Players
Loaded C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project\Football Database\shots.csv into table Shots
Loaded C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project\Football Database\teams.csv into table Teams
Loaded C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project\Football Database\teamstats.csv into table TeamStats


In [57]:
# Verify tables are loaded
print("Tables in the database:")
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print(tables)

Tables in the database:
          name
0  Appearances
1        Games
2      Leagues
3      Players
4        Shots
5        Teams
6    TeamStats


In [58]:
# Load SQL magic
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [59]:
# Configure SQL magic display settings
%config SqlMagic.feedback = False
%config SqlMagic.autopandas = True  # This converts results to pandas DataFrames
%config SqlMagic.displaywidth = 1000  # Controls width

# Add pandas display options to control both width and length
pd.set_option('display.max_rows', 20)     # Limits number of rows
pd.set_option('display.max_columns', 20)  # Limits number of columns
pd.set_option('display.width', 1000)      # Controls overall display width
pd.set_option('display.max_colwidth', 100) # Controls maximum column width

c:\ProgramData\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3284: UserWarning: Config option `displaywidth` not recognized by `SqlMagic`.  Did you mean one of: `displaycon, displaylimit`?
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [60]:
# Connect to the SQLite database file
%sql sqlite:///football_database.db  


In [61]:
%%sql
SELECT COUNT(*)
FROM Leagues


 * sqlite:///football_database.db


,COUNT(*)
0,5


In [44]:
#Using Python to automate queries across all tables as an example
for table_name in csv_files.keys():
    query = f"SELECT COUNT(*) AS row_count FROM {table_name};"
    result = pd.read_sql_query(query, conn)
    print(f"Table {table_name} has {result['row_count'][0]} rows.")

Table Appearances has 356513 rows.
Table Games has 12680 rows.
Table Leagues has 5 rows.
Table Players has 7659 rows.
Table Shots has 324543 rows.
Table Teams has 146 rows.
Table TeamStats has 25360 rows.


In [62]:

%%sql
--Get all teams IDs that played in the Premier League
SELECT DISTINCT homeTeamID as id
FROM games
WHERE leagueID in (SELECT leagueID
                  FROM leagues
                  WHERE leagueID = 1)
                  



 * sqlite:///football_database.db


,id
0,89
1,73
2,72
3,75
4,79
...,...
26,228
27,229
28,227
29,238


In [63]:
%%sql
--Display every team name in the premier league
WITH premierLeagueIDs AS (SELECT DISTINCT homeTeamID as id
FROM games
WHERE leagueID in (SELECT leagueID
                  FROM leagues
                  WHERE leagueID = 1))

SELECT t.teamID, t.name
FROM teams t
WHERE t.teamID in (SELECT id
                   FROM premierLeagueIDs);



 * sqlite:///football_database.db


,teamID,name
0,71,Aston Villa
1,72,Everton
2,74,Southampton
3,75,Leicester
4,76,West Bromwich Albion
...,...,...
26,227,Cardiff
27,228,Fulham
28,229,Wolverhampton Wanderers
29,238,Sheffield United


In [64]:
%%sql
--Calculate how many goals Arsenal scored at home by season
SELECT 
    season, 
    SUM(homegoals) as Total_Home_Goals
        
FROM games g
WHERE g.homeTeamID = 83
GROUP BY season
ORDER BY season;

 * sqlite:///football_database.db


,season,Total_Home_Goals
0,2014,41
1,2015,31
2,2016,39
3,2017,54
4,2018,42
5,2019,36
6,2020,24


In [65]:
%%sql
--Calculate how many goals Arsenal scored at home vs away from the 2014 to 2020 season
SELECT 
    season, 
    SUM(CASE WHEN homeTeamID = 83 THEN homegoals ELSE 0 END) AS Total_Home_Goals,
    SUM(CASE WHEN awayTeamID = 83 THEN awaygoals ELSE 0 END) AS Total_Away_goals
FROM games
WHERE (homeTeamID = 83
    OR awayTeamID = 83)
GROUP BY season
ORDER BY season;

 * sqlite:///football_database.db


,season,Total_Home_Goals,Total_Away_goals
0,2014,41,30
1,2015,31,34
2,2016,39,38
3,2017,54,20
4,2018,42,31
5,2019,36,20
6,2020,24,31


In [66]:
%%sql
/* Shows all winners and losers in 2020 Premier League matches with: */
SELECT  g.season, 
        g.date, 
        (SELECT t.name
         FROM teams t
         WHERE t.teamID = g.homeTeamID
        ) AS Home_Team,
        (SELECT t.name
         FROM teams t
         WHERE t.teamID = g.awayTeamID
        ) AS Away_Team, 
        (CASE WHEN homeGoals > awayGoals THEN "Home Team Wins"
        WHEN homeGoals < awayGoals THEN "Away Team Wins"
        ELSE 'Tie' END) As Winning_Side
FROM games g
WHERE leagueID = 1 and season = '2020'
ORDER BY date DESC;



 * sqlite:///football_database.db


,season,date,Home_Team,Away_Team,Winning_Side
0,2020,2021-05-23 15:00:00,Arsenal,Brighton,Home Team Wins
1,2020,2021-05-23 15:00:00,Aston Villa,Chelsea,Home Team Wins
2,2020,2021-05-23 15:00:00,Fulham,Newcastle United,Away Team Wins
3,2020,2021-05-23 15:00:00,Leeds,West Bromwich Albion,Home Team Wins
4,2020,2021-05-23 15:00:00,Leicester,Tottenham,Away Team Wins
...,...,...,...,...,...
375,2020,2020-09-13 13:00:00,West Bromwich Albion,Leicester,Away Team Wins
376,2020,2020-09-12 19:00:00,West Ham,Newcastle United,Away Team Wins
377,2020,2020-09-12 16:30:00,Liverpool,Leeds,Home Team Wins
378,2020,2020-09-12 14:00:00,Crystal Palace,Southampton,Home Team Wins


In [67]:
%%sql
--Same query as above writen differently
SELECT g.season, 
    g.date, 
    home.name AS HomeTeamName,
    away.name AS AwayTeamName,
    CASE 
        WHEN g.homeGoals > g.awayGoals THEN 'Home Team Wins'
        WHEN g.homeGoals < g.awayGoals THEN 'Away Team Wins'
        ELSE 'TIE' 
    END AS Winning_Side
FROM games g
INNER JOIN teams home ON g.homeTeamID = home.teamID
INNER JOIN teams away ON g.awayTeamID = away.teamID
WHERE g.leagueID = 1
ORDER BY g.date DESC;

 * sqlite:///football_database.db


,season,date,HomeTeamName,AwayTeamName,Winning_Side
0,2020,2021-05-23 15:00:00,Arsenal,Brighton,Home Team Wins
1,2020,2021-05-23 15:00:00,Aston Villa,Chelsea,Home Team Wins
2,2020,2021-05-23 15:00:00,Fulham,Newcastle United,Away Team Wins
3,2020,2021-05-23 15:00:00,Leeds,West Bromwich Albion,Home Team Wins
4,2020,2021-05-23 15:00:00,Leicester,Tottenham,Away Team Wins
...,...,...,...,...,...
2655,2014,2014-08-16 15:00:00,Queens Park Rangers,Hull,Away Team Wins
2656,2014,2014-08-16 15:00:00,Stoke,Aston Villa,Away Team Wins
2657,2014,2014-08-16 15:00:00,West Bromwich Albion,Sunderland,TIE
2658,2014,2014-08-16 15:00:00,West Ham,Tottenham,Away Team Wins


In [68]:
%%sql
-- Calculate the sum of goals scored by each team in the Premier League 2020 and the cumulative sum of goals scored
SELECT home.name AS Home_Team, away.name AS Away_Team, g.homeGoals as Home_Goals, g.awayGoals as Away_Goals, (g.homeGoals + g.awayGoals) AS Total_Goals,
       SUM(g.homeGoals + g.awayGoals) OVER (ORDER BY g.date) AS Cumulative_Goals
FROM games g
LEFT JOIN teams home ON home.teamID = g.homeTeamID
LEFT JOIN teams away ON away.teamID = g.awayTeamID
WHERE g.leagueID = 1 AND g.season = '2020'



 * sqlite:///football_database.db


,Home_Team,Away_Team,Home_Goals,Away_Goals,Total_Goals,Cumulative_Goals
0,Fulham,Arsenal,0,3,3,3
1,Crystal Palace,Southampton,1,0,1,4
2,Liverpool,Leeds,4,3,7,11
3,West Ham,Newcastle United,0,2,2,13
4,West Bromwich Albion,Leicester,0,3,3,16
...,...,...,...,...,...,...
375,Liverpool,Crystal Palace,2,0,2,1024
376,Manchester City,Everton,5,0,5,1024
377,Sheffield United,Burnley,1,0,1,1024
378,West Ham,Southampton,3,0,3,1024


In [69]:
%%sql
/*Calculate the most goals scored by a team in the Premier league 2020 indicating the team name,
the team's side, the opponent name, the date and the maximum goals*/
WITH Max_Goals_Table AS (
    SELECT MAX(homeGoals) AS Max_Goals, homeTeamID AS Team_ID, 'Home' AS Side, date, awayTeamID AS Opponent_ID
    FROM games g
    WHERE g.leagueID =1 AND g.season = '2020'

   UNION ALL

   SELECT MAX(awayGoals) AS Max_Goals, awayTeamID AS Team_ID, 'Away' AS Side, date, homeTeamID AS Opponent_ID
   FROM games g
   WHERE g.leagueID =1 AND g.season = '2020')

SELECT mg.Max_Goals AS 'Maximum Goals in a Game',
       (CASE WHEN mg.Side = 'Home' THEN home.name
       WHEN mg.Side = 'Away' THEN away.name END) AS Team_Name,
       mg.Side, 
       (CASE WHEN mg.Side = 'Home' THEN (SELECT away.name FROM teams away WHERE away.teamID = mg.Opponent_ID)
       WHEN mg.side = 'Away' THEN (SELECT home.name FROM teams home WHERE home.teamID = mg.Opponent_ID) END) AS Opponent_Name,
       strftime('%d-%m-%Y', mg.date) AS Date
FROM Max_Goals_Table mg
LEFT JOIN teams home ON home.teamID = mg.team_ID
LEFT JOIN teams away ON away.teamID = mg.team_ID
WHERE mg.Max_Goals = (SELECT MAX(Max_Goals) FROM Max_Goals_Table)


 * sqlite:///football_database.db


,Maximum Goals in a Game,Team_Name,Side,Opponent_Name,Date
0,9,Manchester United,Home,Southampton,02-02-2021


In [70]:
%%sql
-- Calculate the moving average of goals scored per game for the past 5 games in the 2020 Premier League season
SELECT gameID AS Game_Number,(HomeGoals + AwayGoals) AS Total_Goals, AVG(HomeGoals + AwayGoals) OVER (ORDER BY date ASC ROWS BETWEEN 5 PRECEDING AND CURRENT ROW) AS "5_Game_Moving_Average_Goals"
FROM games 
WHERE leagueID = 1 AND season = '2020'


 * sqlite:///football_database.db


,Game_Number,Total_Goals,5_Game_Moving_Average_Goals
0,14086,3,3.000000
1,14087,1,2.000000
2,14090,7,3.666667
3,14091,2,3.250000
4,14092,3,3.200000
...,...,...,...
375,14810,2,3.166667
376,14811,5,3.666667
377,14812,1,3.333333
378,14813,3,3.500000


In [71]:
%%sql
-- Find the top 10 players with the highest average goals scored per game
SELECT p.name AS Player_Name, ROUND(AVG(app.goals),2) AS Average_Goals, SUM(goals) AS Total_Goals
FROM appearances app
LEFT JOIN players p ON p.playerID = app.playerID
GROUP BY p.playerID 
HAVING SUM(app.goals) IS NOT NULL and SUM(app.goals) > 10
ORDER BY Average_Goals DESC
LIMIT 10;


 * sqlite:///football_database.db


,Player_Name,Average_Goals,Total_Goals
0,Cristiano Ronaldo,0.96,215
1,Lionel Messi,0.95,231
2,Erling Haaland,0.93,40
3,Robert Lewandowski,0.93,203
4,Zlatan Ibrahimovic,0.79,99
5,Luis Suárez,0.75,168
6,Kylian Mbappe-Lottin,0.72,107
7,Harry Kane,0.71,163
8,Sergio Agüero,0.70,132
9,Neymar,0.68,114


In [72]:
%%sql 
-- Calculate the distribution of players across quintiles based on their average goals scored per game
WITH Quintile_Table AS (SELECT p.name AS Player_Name, ROUND(AVG(app.goals),2) AS Average_Goals, SUM(goals) AS Total_Goals, 
       NTILE(5) OVER (ORDER BY AVG(app.goals) DESC) AS Quintile
FROM appearances app
LEFT JOIN players p ON p.playerID = app.playerID
-- Group by playerID as names could potentially have duplicates and grouping by primary key is usually better
GROUP BY p.playerID 
HAVING SUM(app.goals) IS NOT NULL and SUM(app.goals) > 10
ORDER BY Average_Goals DESC)

SELECT Quintile, COUNT(*) AS Number_of_Players, ROUND(AVG(Average_Goals),2) AS Average_Goals 
FROM Quintile_Table
GROUP BY Quintile
ORDER BY Quintile;


 * sqlite:///football_database.db


,Quintile,Number_of_Players,Average_Goals
0,1,171,0.40
1,2,170,0.24
2,3,170,0.18
3,4,170,0.13
4,5,170,0.08


In [73]:
%%sql
-- Getting comprehensive statistics of player performance by 
SELECT 
    p.playerID,
    p.name as Player_Name, 
    t.teamID,
    t.name as Team_Name, 
    l.leagueID,
    l.name as League_Name, 
    g.season,
    COUNT(DISTINCT a.gameID) AS Games_Played, 
    SUM(a.goals) as Total_Goals, 
    SUM(a.assists) as Total_Assists, 
    SUM(a.shots) as Total_Shots, 
    ROUND(CAST(SUM(a.goals) AS FLOAT )/ NULLIF(SUM(a.shots),0), 2) * 100 AS Shots_Converted_Perc,
    ROUND(CAST(SUM(a.goals) AS FLOAT )/COUNT(DISTINCT a.gameID),2) AS Goals_Per_Game
FROM appearances a
LEFT JOIN players p ON p.playerID = a.playerID
LEFT JOIN games g ON g.gameID = a.gameID
LEFT JOIN leagues l ON l.leagueID = g.leagueID
LEFT JOIN teams t ON t.teamID IN (g.homeTeamID, g.awayTeamID)
GROUP BY p.playerID, p.name, t.teamID, t.name, l.leagueID, l.name, g.season
HAVING Games_Played >= 5
ORDER BY Goals_Per_Game DESC;

 * sqlite:///football_database.db


,playerID,Player_Name,teamID,Team_Name,leagueID,League_Name,season,Games_Played,Total_Goals,Total_Assists,Total_Shots,Shots_Converted_Perc,Goals_Per_Game
0,227,Robert Lewandowski,117,Bayern Munich,3,Bundesliga,2020,29,41,7,135,30.0,1.41
1,2371,Cristiano Ronaldo,150,Real Madrid,4,La Liga,2014,35,48,16,225,21.0,1.37
2,1741,Zlatan Ibrahimovic,161,Paris Saint Germain,5,Ligue 1,2015,31,38,13,155,25.0,1.23
3,2098,Luis Suárez,148,Barcelona,4,La Liga,2015,35,40,16,137,29.0,1.14
4,3423,Kylian Mbappe-Lottin,161,Paris Saint Germain,5,Ligue 1,2018,29,33,7,125,26.0,1.14
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15935,9340,Helibelton Palacios,207,Elche,4,La Liga,2020,11,0,0,0,NaN,0.00
15936,9345,Ahmadou Bamba Dieng,164,Marseille,5,Ligue 1,2020,5,0,0,1,0.0,0.00
15937,9419,Bryan Reynolds,95,Roma,2,Serie A,2020,5,0,0,0,NaN,0.00
15938,9427,Mehmet Can Aydin,124,Schalke 04,3,Bundesliga,2020,6,0,0,1,0.0,0.00


In [75]:
%%sql
-- create a query to find Arsenal's ID 
SELECT *
FROM teams
WHERE name LIKE '%Arsenal%';


 * sqlite:///football_database.db


,teamID,name
0,83,Arsenal


In [76]:
%%sql
-- Calculate Arsenal's top scoring players by displaying goals scored, shots taken and the shot/goal ratio by season per player
SELECT p.name, p.playerID, season, SUM(a.goals) AS Goals_Scored, SUM(a.shots) AS Shots_Taken, Round(AVG(a.shots)/AVG(a.goals),3) as "Avg Shots Taken per Goal"
FROM appearances a 
LEFT JOIN games g ON a.gameID = g.gameID
LEFT JOIN teams t ON t.teamID IN (g.homeTeamID, g.awayTeamID)
LEFT JOIN players p ON p.playerID = a.playerID 
WHERE t.teamID = 83 
GROUP BY a.playerID, g.season
ORDER BY Goals_scored DESC
LIMIT 10;


 * sqlite:///football_database.db


,name,playerID,season,Goals_Scored,Shots_Taken,Avg Shots Taken per Goal
0,Alexis Sánchez,498,2016,24,129,5.375
1,Pierre-Emerick Aubameyang,318,2018,22,94,4.273
2,Pierre-Emerick Aubameyang,318,2019,22,93,4.227
3,Alexis Sánchez,498,2014,16,122,7.625
4,Olivier Giroud,502,2015,16,101,6.313
5,Olivier Giroud,502,2014,14,70,5.000
6,Alexandre Lacazette,3277,2017,14,67,4.786
7,Alexis Sánchez,498,2015,13,107,8.231
8,Alexandre Lacazette,3277,2018,13,81,6.231
9,Alexandre Lacazette,3277,2020,13,45,3.462


In [77]:
%%sql
-- Rank the teams according to the number of games won over 2014 to 2020 seasons for premier league teams in the dataset 
WITH premMatches AS (SELECT  g.season, 
        g.date, 
        (SELECT t.name
         FROM teams t
         WHERE t.teamID = g.homeTeamID
        ) AS Home_Team,
        (SELECT t.name
         FROM teams t
         WHERE t.teamID = g.awayTeamID
        ) AS Away_Team, 
        (CASE WHEN homeGoals > awayGoals THEN "Home Team Wins"
        WHEN homeGoals < awayGoals THEN "Away Team Wins"
        ELSE 'Tie' END) As Winning_Side
FROM games g
WHERE leagueID = 1 
ORDER BY date DESC)

SELECT TEAM, SUM(Wins) AS "Total Wins", RANK () OVER (ORDER BY SUM(Wins) DESC) AS "Rank"
FROM (

SELECT Home_Team AS Team, COUNT(*) As Wins
FROM premMatches 
WHERE Winning_Side = "Home Team Wins"
GROUP BY Home_Team

UNION ALL

SELECT Away_Team AS TEAM, COUNT(*) As Wins
FROM premMatches 
WHERE Winning_Side = "Away Team Wins"
GROUP BY Away_Team
) AS Combined_Wins
GROUP BY TEAM
ORDER BY "Total Wins" DESC
LIMIT 10;


 * sqlite:///football_database.db


,Team,Total Wins,Rank
0,Manchester City,183,1
1,Liverpool,159,2
2,Chelsea,149,3
3,Tottenham,144,4
4,Manchester United,140,5
5,Arsenal,137,6
6,Leicester,111,7
7,Everton,98,8
8,West Ham,94,9
9,Southampton,91,10


In [78]:
%%sql 
-- Calculate the points for each team in the database
SELECT l.name AS League,
       g.season,
       t.name AS Team,
       Count(*) AS Games_Played,
       COUNT((CASE 
            --Home teams wins
            WHEN homeGoals > awayGoals AND g.homeTeamID = t.teamID THEN 1 
            --Away teams wins
            WHEN homeGoals < awayGoals AND g.awayTeamID = t.teamID THEN 1 END)) * 3 +
       --Tie
       COUNT(CASE WHEN homeGoals = awayGoals THEN 1 END) AS Points, 
       -- Count all Wins
       COUNT(CASE 
            WHEN homeGoals > awayGoals AND g.homeTeamID = t.teamID THEN 1 
            WHEN homeGoals < awayGoals AND g.awayTeamID = t.teamID THEN 1 END)  AS Wins,
       COUNT(CASE 
            WHEN homeGoals < awayGoals AND g.homeTeamID = t.teamID THEN 1 
            WHEN homeGoals > awayGoals AND g.awayTeamID = t.teamID THEN 1 END)  AS Losses, 
       COUNT(CASE WHEN homeGoals = awayGoals THEN 1 END) AS Draws
          
FROM games g 
LEFT JOIN leagues l ON l.leagueID = g.leagueID
LEFT JOIN teams t ON t.teamID IN (g.homeTeamID, g.awayTeamID)
--Add the following for a specific league and season WHERE l.name = 'Premier League' AND g.season = '2020'
GROUP BY l.leagueID, g.season, t.name 
ORDER BY Points DESC


 * sqlite:///football_database.db


,League,season,Team,Games_Played,Points,Wins,Losses,Draws
0,Premier League,2017,Manchester City,38,100,32,2,4
1,Premier League,2019,Liverpool,38,99,32,3,3
2,Premier League,2018,Manchester City,38,98,32,4,2
3,Premier League,2018,Liverpool,38,97,30,1,7
4,Ligue 1,2015,Paris Saint Germain,38,96,30,2,6
...,...,...,...,...,...,...,...,...
681,Premier League,2015,Aston Villa,38,17,3,27,8
682,Premier League,2018,Huddersfield,38,16,3,28,7
683,Bundesliga,2020,Schalke 04,34,16,3,24,7
684,Serie A,2016,Pescara,38,15,2,27,9


In [79]:
%%sql 
-- Calculates detailed team performance metrics including points, goals scored/conceded, and points per game
-- for all teams across all seasons and leagues to analyze overall team performance
WITH season_points AS (SELECT 
    l.name AS League,
    g.season,
    t.name AS Team,
    COUNT((CASE 
        --Home teams wins
        WHEN homeGoals > awayGoals AND g.homeTeamID = t.teamID THEN 1 
        --Away teams wins
        WHEN homeGoals < awayGoals AND g.awayTeamID = t.teamID THEN 1 END)) * 3 +
    --Tie
    COUNT(CASE WHEN homeGoals = awayGoals THEN 1 END) AS Points, 
    COUNT (*) as Games_Played, 
    SUM(CASE WHEN g.homeTeamID = t.teamID THEN homeGoals
             WHEN g.awayTeamID = t.teamID THEN awayGoals END) as Goals_For,
    SUM(CASE WHEN g.homeTeamID = t.teamID THEN awayGoals
             WHEN g.awayTeamID = t.teamID THEN homeGoals END) as Goals_Against
FROM games g 
LEFT JOIN leagues l ON l.leagueID = g.leagueID
LEFT JOIN teams t ON t.teamID IN (g.homeTeamID, g.awayTeamID)
GROUP BY l.leagueID, g.season, t.name
)

SELECT *, Goals_For - Goals_Against as Goal_Difference, 
    ROUND(CAST(points AS FLOAT)/ Games_Played, 2) As Points_Per_Game
FROM season_points

 * sqlite:///football_database.db


,League,season,Team,Points,Games_Played,Goals_For,Goals_Against,Goal_Difference,Points_Per_Game
0,Premier League,2014,Arsenal,75,38,71,36,35,1.97
1,Premier League,2014,Aston Villa,38,38,31,57,-26,1.00
2,Premier League,2014,Burnley,33,38,28,53,-25,0.87
3,Premier League,2014,Chelsea,87,38,73,32,41,2.29
4,Premier League,2014,Crystal Palace,48,38,47,51,-4,1.26
...,...,...,...,...,...,...,...,...,...
681,Ligue 1,2020,Paris Saint Germain,82,38,86,28,58,2.16
682,Ligue 1,2020,Reims,42,38,42,50,-8,1.11
683,Ligue 1,2020,Rennes,58,38,52,40,12,1.53
684,Ligue 1,2020,Saint-Etienne,46,38,42,54,-12,1.21


In [80]:
%%sql 
--Display premier league winning teams showing reiging champions

WITH Season_Points AS (
    SELECT season, t.name AS Team_Name,
        SUM((CASE 
            WHEN g.hometeamID = t.teamID AND g.homeGoals > g.awayGoals THEN 3 
            WHEN g.awayteamID = t.teamID AND g.homeGoals < g.awayGoals THEN 3 END)) + 
        SUM((CASE 
            WHEN g.homeGoals = g.awayGoals THEN 1 END)) AS Season_Points
FROM games g
LEFT JOIN teams t ON t.teamID IN (g.homeTeamID, g.awayTeamID)
WHERE g.leagueID = 1
GROUP BY t.teamID, season
ORDER BY Season_Points DESC), 

Season_Winners AS (
    -- Identify the winner for each season 
    SELECT season, Team_Name, 
        ROW_NUMBER() OVER(PARTITION BY season ORDER BY Season_Points DESC) AS Season_Rank
    FROM Season_Points 
    GROUP BY season
)

SELECT  
    sw.season, 
    sw.Team_Name AS Champion, 
    LAG(sw.Team_Name, 1) OVER(ORDER BY season) AS Previous_Champion,
    (CASE WHEN sw.Team_Name = (LAG(sw.Team_Name, 1) OVER(ORDER BY season)) THEN 'Yes' ELSE 'No' END) AS Two_Time_Champion
FROM Season_Winners sw
WHERE sw.Season_Rank = 1 
GROUP BY sw.season
ORDER BY sw.season ASC;

 * sqlite:///football_database.db


,season,Champion,Previous_Champion,Two_Time_Champion
0,2014,Chelsea,None,No
1,2015,Leicester,Chelsea,No
2,2016,Chelsea,Leicester,No
3,2017,Manchester City,Chelsea,No
4,2018,Manchester City,Manchester City,Yes
5,2019,Liverpool,Manchester City,No
6,2020,Manchester City,Liverpool,No


In [81]:
%%sql 
-- Analyzes league competitiveness by calculating average points gap between top/bottom teams
-- Lower points gap indicates a more competitive league

WITH season_points AS (SELECT l.name AS League,
       g.season,
       t.name AS Team,
       COUNT((CASE 
            --Home teams wins
            WHEN homeGoals > awayGoals AND g.homeTeamID = t.teamID THEN 1 
            --Away teams wins
            WHEN homeGoals < awayGoals AND g.awayTeamID = t.teamID THEN 1 END)) * 3 +
       --Tie
       COUNT(CASE WHEN homeGoals = awayGoals THEN 1 END) AS Points 
FROM games g 
LEFT JOIN leagues l ON l.leagueID = g.leagueID
LEFT JOIN teams t ON t.teamID IN (g.homeTeamID, g.awayTeamID)
GROUP BY l.leagueID, g.season, t.name 
ORDER BY Points DESC)

SELECT league, 
       ROUND(AVG(Points_GAP),2) AS Avg_Points_Gap,  -- Average gap across seasons
       ROW_NUMBER() OVER (ORDER BY AVG(Points_Gap) DESC) AS Rank
FROM(
     SELECT League, Season, Max(Points) - Min(Points) AS Points_Gap -- Gap per season
     FROM season_points
     GROUP BY League, Season) Sub
GROUP BY League
ORDER BY Avg_Points_Gap DESC

 * sqlite:///football_database.db


,League,Avg_Points_Gap,Rank
0,Premier League,68.86,1
1,Serie A,68.57,2
2,La Liga,64.57,3
3,Ligue 1,63.00,4
4,Bundesliga,59.00,5


In [82]:
%%sql 
-- Analyze league competitiveness by comparing average points and points gaps between top and bottom teams across leagues
WITH season_points AS (
    SELECT 
        l.leagueID,
        l.name AS League,
        g.season,
        t.teamID,
        t.name AS Team,
        COUNT((CASE 
            --Home teams wins
            WHEN homeGoals > awayGoals AND g.homeTeamID = t.teamID THEN 1 
            --Away teams wins
            WHEN homeGoals < awayGoals AND g.awayTeamID = t.teamID THEN 1 END)) * 3 +
        --Tie
        COUNT(CASE WHEN homeGoals = awayGoals THEN 1 END) AS Points 
    FROM games g 
    LEFT JOIN leagues l ON l.leagueID = g.leagueID
    LEFT JOIN teams t ON t.teamID IN (g.homeTeamID, g.awayTeamID)
    GROUP BY l.leagueID, l.name, g.season, t.teamID, t.name),

league_stats AS (
    SELECT
        league,
        leagueID, 
        season,
        MAX(points) - MIN(points) AS Points_Gap_Top_to_Bottom,
        ROUND(AVG(points),2) AS Avg_Points,
        AVG(CASE WHEN Standing <= 4 THEN points END) AS Top_4_Teams_Avg_Points,
        AVG(CASE WHEN Standing >= (SELECT COUNT(*) - 3 FROM season_points sp2 
                                  WHERE sp2.league = season_standings.league 
                                  AND sp2.season = season_standings.season) 
            THEN points END) AS Bottom_4_Teams_Avg_Points
    FROM (
        SELECT 
            league,
            leagueID,
            season, 
            team,
            teamID,
            points,
            ROW_NUMBER() OVER (PARTITION BY League, season ORDER BY points DESC) as Standing
        FROM season_points
    ) AS Season_Standings
    GROUP BY league, leagueID, season
)

SELECT 
    league,
    leagueID,
    season,
    ROUND(AVG(Avg_Points),2) AS Avg_Points,
    ROUND(AVG(Points_Gap_Top_to_Bottom),2) AS Avg_Points_Gap_Top_to_Bottom,
    ROUND(AVG(Top_4_Teams_Avg_Points - Bottom_4_Teams_Avg_Points),2) AS Points_Gap_Between_Top_and_Bottom_4_Teams    
FROM league_stats
GROUP BY league, leagueID, season
ORDER BY Avg_Points DESC;

 * sqlite:///football_database.db


,league,leagueID,season,Avg_Points,Avg_Points_Gap_Top_to_Bottom,Points_Gap_Between_Top_and_Bottom_4_Teams
0,Premier League,1,2018,53.45,82.0,56.50
1,Serie A,2,2016,53.00,76.0,57.25
2,Premier League,1,2020,52.85,63.0,45.00
3,Serie A,2,2017,52.85,74.0,54.00
4,Premier League,1,2016,52.80,69.0,51.75
...,...,...,...,...,...,...
30,Bundesliga,3,2016,46.89,57.0,36.00
31,Bundesliga,3,2020,46.50,62.0,38.25
32,Bundesliga,3,2014,46.44,48.0,35.00
33,Bundesliga,3,2017,46.39,62.0,33.75


In [84]:
# This cell analyzes the points distribution across leagues and seasons
# First, query the database to get total points for each team per season
stats_df = pd.read_sql_query("""
WITH season_points AS (
    SELECT l.name AS League,
           g.season,
           t.name AS Team,
           COUNT((CASE 
                WHEN homeGoals > awayGoals AND g.homeTeamID = t.teamID THEN 1 
                WHEN homeGoals < awayGoals AND g.awayTeamID = t.teamID THEN 1 END)) * 3 +
           COUNT(CASE WHEN homeGoals = awayGoals THEN 1 END) AS Points 
    FROM games g 
    LEFT JOIN leagues l ON l.leagueID = g.leagueID
    LEFT JOIN teams t ON t.teamID IN (g.homeTeamID, g.awayTeamID)
    GROUP BY l.leagueID, g.season, t.name 
    ORDER BY Points DESC)
SELECT * FROM season_points;
""", conn)

# Calculate key statistics for each league/season:
# - Total points across all teams
# - Average points per team 
# - Standard deviation of points (measure of competitiveness)
stats = stats_df.groupby(['League', 'season'])['Points'].agg([
    ('Total_Points', 'sum'),
    ('Avg_Points', 'mean'),
    ('Std_Points', 'std')
]).round(2)

# Calculate the points gap between top and bottom teams
# This helps identify leagues with big disparities between teams
points_range = stats_df.groupby(['League', 'season'])['Points'].agg(['min', 'max'])
stats['Points_Gap'] = points_range['max'] - points_range['min']

# Sort leagues by average points to see which are highest scoring
stats = stats.sort_values('Avg_Points', ascending=False)

# Show the final analysis 
display(stats)

Total_Points  Avg_Points  Std_Points  Points_Gap
League         season                                                  
Premier League 2018            1069       53.45       21.01          82
Serie A        2016            1060       53.00       20.80          76
               2017            1057       52.85       20.55          74
Premier League 2020            1057       52.85       16.89          63
               2016            1056       52.80       19.85          69
...                             ...         ...         ...         ...
Bundesliga     2016             844       46.89       13.94          57
               2020             837       46.50       15.11          62
               2014             836       46.44       13.59          48
               2017             835       46.39       13.99          62
Ligue 1        2019             767       38.35       11.81          55

[35 rows x 4 columns]

In [85]:
%%sql
--Analyzing team stats per season per team for all leagues in database
WITH team_performance AS (SELECT 
    l.leagueID,
    l.name AS League, 
    ts.season, 
    t.name, 
    t.teamID,
    COUNT(*) AS Games_Played, 
    COUNT(CASE WHEN ts.result = 'W' THEN 1 END) as Wins,
    COUNT(CASE WHEN ts.result = 'D' THEN 1 END) as Draws,
    COUNT(CASE WHEN ts.result = 'L' THEN 1 END) as Losses,
    SUM(CASE WHEN g.homeTeamID = t.teamID THEN homeGoals
            WHEN g.awayTeamID = t.teamID THEN awayGoals END) as Goals_Scored,
    SUM(CASE WHEN g.homeTeamID = t.teamID THEN awayGoals
            WHEN g.awayTeamID = t.teamID THEN homeGoals END) as Goals_Conceded,
    SUM(ts.shots) AS Total_Shots,
    SUM(ts.shotsOnTarget) AS Total_Shots_On_Target, 
    SUM(ts.fouls) AS Total_Fouls, 
    SUM(ts.corners) as Total_Corners,
    SUM(ts.yellowcards) as Total_Yellow_Cards,
    SUM(ts.redcards) as Total_Red_Cards
FROM games g
LEFT JOIN leagues l on l.leagueID = g.leagueID
LEFT JOIN teams t ON t.teamID IN (g.homeTeamID, g.awayTeamID)
LEFT JOIN teamstats ts ON ts.gameID = g.gameID -- Match the game records first 
    -- Match the teamID filed in teamstats when it is either the home team or the away team 
    AND ((ts.teamID = t.teamID and g.homeTeamID = ts.teamID)
    OR (ts.teamID = t.teamID and g.awayTeamID = ts.teamID))
GROUP BY l.name, ts.season, t.name
)

SELECT *, 
    ROUND(1.0 * Goals_Scored/ Games_Played, 2) AS Goals_Per_Game, 
    ROUND (Total_Shots_On_Target / NULLIF (Goals_Scored, 0),2) AS Shots_Per_Goal
FROM team_performance
ORDER BY season ASC, Goals_Scored DESC;


 * sqlite:///football_database.db


,leagueID,League,season,name,teamID,Games_Played,Wins,Draws,Losses,Goals_Scored,Goals_Conceded,Total_Shots,Total_Shots_On_Target,Total_Fouls,Total_Corners,Total_Yellow_Cards,Total_Red_Cards,Goals_Per_Game,Shots_Per_Goal
0,4,La Liga,2014,Real Madrid,150,38,30,2,6,118,38,686,283,407,246,85.0,3,3.11,2.0
1,4,La Liga,2014,Barcelona,148,38,30,4,4,110,21,625,273,369,233,66.0,3,2.89,2.0
2,5,Ligue 1,2014,Paris Saint Germain,161,38,24,11,3,83,36,481,219,440,171,63.0,3,2.18,2.0
3,1,Premier League,2014,Manchester City,88,38,24,7,7,83,38,670,229,445,273,77.0,2,2.18,2.0
4,3,Bundesliga,2014,Bayern Munich,117,34,25,4,5,80,18,585,235,414,222,37.0,2,2.35,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681,1,Premier League,2020,Fulham,228,38,5,13,20,27,53,442,135,484,164,66.0,3,0.71,5.0
682,3,Bundesliga,2020,Arminia Bielefeld,262,34,9,8,17,26,52,334,102,391,118,51.0,1,0.76,3.0
683,3,Bundesliga,2020,Schalke 04,124,34,3,7,24,25,86,302,85,417,124,74.0,2,0.74,3.0
684,5,Ligue 1,2020,Dijon,181,38,4,9,25,25,73,350,93,513,146,77.0,5,0.66,3.0


In [46]:
#Exporting important SQL queries to csv files for data visualization in Tableau next 

# Player Performance Analysis Table
player_query = """
SELECT 
    p.playerID,
    p.name as Player_Name, 
    t.teamID,
    t.name as Team_Name, 
    l.leagueID,
    l.name as League_Name, 
    g.season,
    COUNT(DISTINCT a.gameID) AS Games_Played, 
    SUM(a.goals) as Total_Goals, 
    SUM(a.assists) as Total_Assists, 
    SUM(a.shots) as Total_Shots, 
    ROUND(CAST(SUM(a.goals) AS FLOAT )/ NULLIF(SUM(a.shots),0), 2) * 100 AS Shots_Converted_Perc,
    ROUND(CAST(SUM(a.goals) AS FLOAT )/COUNT(DISTINCT a.gameID),2) AS Goals_Per_Game
FROM appearances a
LEFT JOIN players p ON p.playerID = a.playerID
LEFT JOIN games g ON g.gameID = a.gameID
LEFT JOIN leagues l ON l.leagueID = g.leagueID
LEFT JOIN teams t ON t.teamID IN (g.homeTeamID, g.awayTeamID)
GROUP BY p.playerID, p.name, t.teamID, t.name, l.leagueID, l.name, g.season
HAVING Games_Played >= 5
ORDER BY Goals_Per_Game DESC;
"""

# Arsenal Scorer's Analysis Table
arsenal_scorers_query = """
SELECT p.name, p.playerID, season, SUM(a.goals) AS Goals_Scored, SUM(a.shots) AS Shots_Taken, Round(AVG(a.shots)/AVG(a.goals),3) as "Avg Shots Taken per Goal"
FROM appearances a 
LEFT JOIN games g ON a.gameID = g.gameID
LEFT JOIN teams t ON t.teamID IN (g.homeTeamID, g.awayTeamID)
LEFT JOIN players p ON p.playerID = a.playerID 
WHERE t.teamID = 83 
GROUP BY a.playerID, g.season
ORDER BY Goals_scored DESC
LIMIT 10;
"""

# Team Stat Performance Analysis Table
team_stat_performance = """ 
WITH team_performance AS (SELECT 
    l.leagueID,
    l.name AS League, 
    ts.season, 
    t.name, 
    t.teamID,
    COUNT(*) AS Games_Played, 
    COUNT(CASE WHEN ts.result = 'W' THEN 1 END) as Wins,
    COUNT(CASE WHEN ts.result = 'D' THEN 1 END) as Draws,
    COUNT(CASE WHEN ts.result = 'L' THEN 1 END) as Losses,
    SUM(CASE WHEN g.homeTeamID = t.teamID THEN homeGoals
            WHEN g.awayTeamID = t.teamID THEN awayGoals END) as Goals_Scored,
    SUM(CASE WHEN g.homeTeamID = t.teamID THEN awayGoals
            WHEN g.awayTeamID = t.teamID THEN homeGoals END) as Goals_Conceded,
    SUM(ts.shots) AS Total_Shots,
    SUM(ts.shotsOnTarget) AS Total_Shots_On_Target, 
    SUM(ts.fouls) AS Total_Fouls, 
    SUM(ts.corners) as Total_Corners,
    SUM(ts.yellowcards) as Total_Yellow_Cards,
    SUM(ts.redcards) as Total_Red_Cards
FROM games g
LEFT JOIN leagues l on l.leagueID = g.leagueID
LEFT JOIN teams t ON t.teamID IN (g.homeTeamID, g.awayTeamID)
LEFT JOIN teamstats ts ON ts.gameID = g.gameID -- Match the game records first 
    -- Match the teamID filed in teamstats when it is either the home team or the away team 
    AND ((ts.teamID = t.teamID and g.homeTeamID = ts.teamID)
    OR (ts.teamID = t.teamID and g.awayTeamID = ts.teamID))
GROUP BY l.name, ts.season, t.name
)

SELECT *, 
    ROUND(1.0 * Goals_Scored/ Games_Played, 2) AS Goals_Per_Game, 
    ROUND (Total_Shots_On_Target / NULLIF (Goals_Scored, 0),2) AS Shots_Per_Goal
FROM team_performance
ORDER BY season ASC, Goals_Scored DESC;
"""

# Export each query
for name, query in [('player_performance', player_query),
                   ('Arsenal_scorers', arsenal_scorers_query),
                   ('team_stat_performance', team_stat_performance)
                   ]:
    df = pd.read_sql_query(query, conn)
    output_path = f"C:\\Users\\Zaki Bouaoudia\\OneDrive\\09_Programming\\SQL Project\\EU Football SQL Project\\Tableau Exports\\{name}.csv"
    # Explicitly specify the mode as 'w' to overwrite
    df.to_csv(output_path, index=False, mode='w')




In [47]:
conn = sqlite3.connect('multi_table_database.db')
# tables are already saved if using this connection
conn.close()
